# Tailscale Installation Script Security Audits

The corresponding notebook/post includes directions to download and run installation scripts, but I'm always a little wary of running shell scripts I haven't looked over, even if they're from a reliable-seeming source. So I'll scan over install scripts

### Tailscale Client Installation on a Ubuntu/Debian Linux Machine {.linux_install}

```console
curl -fsSL https://tailscale.com/install.sh | sh
```

In [87]:
from IPython.display import display, HTML
import requests

In [2]:
url = "https://tailscale.com/install.sh"
resp = requests.get(url)

In [3]:
resp.raise_for_status()
linux_script_lines = resp.text
print(linux_script_lines)

#!/bin/sh
# Copyright (c) Tailscale Inc & AUTHORS
# SPDX-License-Identifier: BSD-3-Clause
#
# This script detects the current operating system, and installs
# Tailscale according to that OS's conventions.

set -eu

# All the code is wrapped in a main function that gets called at the
# bottom of the file, so that a truncated partial download doesn't end
# up executing half a script.
main() {
	# Step 1: detect the current linux distro, version, and packaging system.
	#
	# We rely on a combination of 'uname' and /etc/os-release to find
	# an OS name and version, and from there work out what
	# installation method we should be using.
	#
	# The end result of this step is that the following three
	# variables are populated, if detection was successful.
	OS=""
	VERSION=""
	PACKAGETYPE=""
	APT_KEY_TYPE="" # Only for apt-based distros
	APT_SYSTEMCTL_START=false # Only needs to be true for Kali
	TRACK="${TRACK:-stable}"

	case "$TRACK" in
		stable|unstable)
			;;
		*)
			echo "unsupported track 

The install script defines and invokes one function, `main()`.

The `main()` function starts off by defining variables for the invoking-machine's:
* OS,
* OS version,
* OS package manager type,
* package signing key type (for distros using the `apt` package manager, e.g. Debian/Ubuntu; options: "legacy" or "keyring"),
* a boolean flag so that `systemctl` commands can be run to enable tailscale to start on boot (only necessary on the Kali linux distro), and
* a `TRACK` variable that only accepts values "stable" or "unstable" (it defaults to "stable" if there's not already a `TRACK` env-var set, so I assume this is just to make it more ergonomic for devs to install nightly releases).

The first four are assigned empty-string placeholder values.

In [8]:
print("\n".join(linux_script_lines.split("\n")[12:36]))

main() {
	# Step 1: detect the current linux distro, version, and packaging system.
	#
	# We rely on a combination of 'uname' and /etc/os-release to find
	# an OS name and version, and from there work out what
	# installation method we should be using.
	#
	# The end result of this step is that the following three
	# variables are populated, if detection was successful.
	OS=""
	VERSION=""
	PACKAGETYPE=""
	APT_KEY_TYPE="" # Only for apt-based distros
	APT_SYSTEMCTL_START=false # Only needs to be true for Kali
	TRACK="${TRACK:-stable}"

	case "$TRACK" in
		stable|unstable)
			;;
		*)
			echo "unsupported track $TRACK"
			exit 1
			;;
	esac


The next block of code checks the invoking-machine for an `/etc/os-release` file, and if present, it "runs" the file, adding all of the file's variables to the shell's present context and updates the above placeholder variables based on the OS indicated by the file's `ID` variable.

If no `/etc/os-release is found, this block is skipped.

$$
f_{\text{OS}}(\text{ID}) =
\begin{cases}
\text{Ubuntu, if ID} \in \{\text{ubuntu, pop, neon, zorin, tuxedo, elementary, galliumos} \} \\
\text{Ubuntu, if ID} = \text{linuxmint } \& \text{ } [\text{UBUNTU\_CODENAME} \neq \text{""} \lor \text{ DEBIAN\_CODENAME} = \text{""}] \\
\text{Ubuntu, if ID} = \text{pika } \& \text{ VERSION\_ID} \lt 4 \\
\text{Debian, if ID} \in \{\text{debian, parrot, mendel, pureos, kaisen, kali, Deepin, deepin, osmc} \} \\
\text{Debian, if ID} = \text{linuxmint } \& \text{ DEBIAN\_CODENAME} \neq \text{""} \\
\text{Debian, if ID} = \text{pika } \& \text{ VERSION\_ID} \geq 4 \\
\text{Raspbian, if ID} \in \{\text{raspbian} \} \\
\text{CentOS, if ID} \in \{\text{centos} \} \\
\text{Oracle, if ID} \in \{\text{ol} \} \\
\text{Red Hat, if ID} \in \{\text{rhel} \} \\
\text{Fedora, if ID} \in \{\text{fedora, rocky, almalinux, nobara, openmandriva, sangoma, risios, cloudlinux, alinux, fedora-asahi-remix} \} \\
\text{Amazon-Linux, if ID} \in \{\text{amzn} \} \\
\text{CentOS, if ID} \in \{\text{xenenterprise} \} \\
\text{OpenSUSE, if ID} \in \{\text{opensuse-leap, sles, opensuse-tumbleweed, sle-micro-rancher} \} \\
\text{Arch, if ID} \in \{\text{arch, archarm, endeavouros, blendos, garuda, archcraft, cachyos} \} \\
\text{Manjaro, if ID} \in \{\text{manjaro, manjaro-arm} \} \\
\text{Alpine, if ID} \in \{\text{alpine, postmarketos} \} \\
\text{Void, if ID} \in \{\text{void} \} \\
\text{Gentoo, if ID} \in \{\text{gentoo} \} \\
\text{FreeBSD, if ID} \in \{\text{freebsd} \} \\
\text{Photon, if ID} \in \{\text{photon} \} \\
\end{cases}
$$

If $\text{ID} = \text{nixos}$, it prints a message directing the user to manually add Tailscale to their `NixOS` config and terminates the script early.

$$
f_{\text{PACKAGETYPE}}(\text{OS, ID}) =
\begin{cases}
\text{apt, if OS} \in \{ \text{Ubuntu, Debian} \} \\
\text{dnf, if OS} \in \{ \text{Fedora} \} \\
\text{dnf, if OS} \in \{ \text{CentOS, Oracle, Red Hat} \} \text{ } \& \text{ } [\lfloor \text{VERSION\_ID} \rfloor \neq 7] \text{ } \& \text{ ID} \neq \text{xenenterprise} \\
\text{yum, if OS} \in \{ \text{CentOS, Oracle, Red Hat} \} \text{ } \& \lfloor \text{VERSION\_ID} \rfloor = 7] \\
\text{yum, if OS} \in \{ \text{Amazon-Linux} \} \lor \text{ID} = \text{xenenterprise} \\
\text{yum, if OS} \in \{ \text{OpenSUSE} \} \\
\text{pacman, if OS} \in \{ \text{Arch, Manjaro} \} \\
\text{apk, if OS} \in \{ \text{Alpine} \} \\
\text{xbps, if OS} \in \{ \text{Void} \} \\
\text{emerge, if OS} \in \{ \text{Gentoo} \} \\
\text{pkg, if OS} \in \{ \text{FreeBSD} \} \\
\text{tdnf, if OS} \in \{ \text{Photon} \} \\
\end{cases}
$$

It also unpacks `VERSION_ID`s and sets the `APT_KEY_TYPE` for Ubuntu/Debian based distros. I'll use python for representing the `APT_KEY_TYPE` logic; $\LaTeX$ is rather cumbersome and not as pretty as I used to believe.

```python
APT_KEY_TYPE = "keyring"

if (
    (ID in ["ubuntu", "pop", "neon", "zorin", "tuxedo"] and not str(VERSION_ID).startswith("2"))
    or (ID in ["debian", "raspbian"] and VERSION_ID < 11)
    or (ID in ["linuxmint", "parrot", "mendel"] and VERSION_ID < 5)
    or (ID in ["elementary"] and VERSION_ID < 6)
    or (ID in ["kali"] and int(str(VERSION_ID)[0:4]) < 2021)
    or (ID in ["Deepin", "deepin"] and VERSION_ID < 20)
    or (ID in ["galliumos"])
):
    APT_KEY_TYPE = "legacy"
```

Ignoring the wild variety of Linux distros, everything so far seems straightforward.

In [19]:
#| code-fold: show
#| code-summary: Showing my /etc/os-release file's contents

!cat /etc/os-release

PRETTY_NAME="Ubuntu 22.04.5 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.5 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [55]:
#| code-fold: show
#| code-summary: The /etc/os-release parsing conditional block
print("\n".join(linux_script_lines.split("\n")[37:304]))

	if [ -f /etc/os-release ]; then
		# /etc/os-release populates a number of shell variables. We care about the following:
		#  - ID: the short name of the OS (e.g. "debian", "freebsd")
		#  - VERSION_ID: the numeric release version for the OS, if any (e.g. "18.04")
		#  - VERSION_CODENAME: the codename of the OS release, if any (e.g. "buster")
		#  - UBUNTU_CODENAME: if it exists, use instead of VERSION_CODENAME
		. /etc/os-release
		case "$ID" in
			ubuntu|pop|neon|zorin|tuxedo)
				OS="ubuntu"
				if [ "${UBUNTU_CODENAME:-}" != "" ]; then
				    VERSION="$UBUNTU_CODENAME"
				else
				    VERSION="$VERSION_CODENAME"
				fi
				PACKAGETYPE="apt"
				# Third-party keyrings became the preferred method of
				# installation in Ubuntu 20.04.
				if expr "$VERSION_ID" : "2.*" >/dev/null; then
					APT_KEY_TYPE="keyring"
				else
					APT_KEY_TYPE="legacy"
				fi
				;;
			debian)
				OS="$ID"
				VERSION="$VERSION_CODENAME"
				PACKAGETYPE="apt"
				# Third-party keyrings became the prefer

If there wasn't a `/etc/os-release` file and the conditional block above was skipped, the `OS` variable will be empty and the shell will enter this block (unless the running context's `env`ironment already set an `OS` variable).

In the block, it checks if the invoking-shell has a command named `uname` (ie checking if the invoking-machine is running Linux/Unix or if it's a Windows machine), and sets the `OS`, `VERSION`, and `PACKAGETYPE` variables based on the string `uname` returns.

Straightforward.

In [56]:
#| code-fold: show
#| code-summary: Showing the output of the uname command
!uname

Linux


In [58]:
#| code-fold: show
#| code-summary: Showing the output of command `uname`
!type uname

uname is /usr/bin/uname


In [66]:
#| code-fold: show
#| code-summary: The conditional block that runs after the /etc/os-release block
print("\n".join(linux_script_lines.split("\n")[305:336]))

	# If we failed to detect something through os-release, consult
	# uname and try to infer things from that.
	if [ -z "$OS" ]; then
		if type uname >/dev/null 2>&1; then
			case "$(uname)" in
				FreeBSD)
					# FreeBSD before 12.2 doesn't have
					# /etc/os-release, so we wouldn't have found it in
					# the os-release probing above.
					OS="freebsd"
					VERSION="$(freebsd-version | cut -f1 -d.)"
					PACKAGETYPE="pkg"
					;;
				OpenBSD)
					OS="openbsd"
					VERSION="$(uname -r)"
					PACKAGETYPE=""
					;;
				Darwin)
					OS="macos"
					VERSION="$(sw_vers -productVersion | cut -f1-2 -d.)"
					PACKAGETYPE="appstore"
					;;
				Linux)
					OS="other-linux"
					VERSION=""
					PACKAGETYPE=""
					;;
			esac
		fi
	fi


This block determines whether `curl` or `wget` are available on the invoking-machine, configures the prefix of web-request-making commands (in the `CURL` variable), and then 

In [71]:
#| code-fold: show
#| code-summary: The conditional block that runs after the /etc/os-release block
print("\n".join(linux_script_lines.split("\n")[337:361]))

	# Ideally we want to use curl, but on some installs we
	# only have wget. Detect and use what's available.
	CURL=
	if type curl >/dev/null; then
		CURL="curl -fsSL"
	elif type wget >/dev/null; then
		CURL="wget -q -O-"
	fi
	if [ -z "$CURL" ]; then
		echo "The installer needs either curl or wget to download files."
		echo "Please install either curl or wget to proceed."
		exit 1
	fi

	TEST_URL="https://pkgs.tailscale.com/"
	RC=0
	TEST_OUT=$($CURL "$TEST_URL" 2>&1) || RC=$?
	if [ $RC != 0 ]; then
		echo "The installer cannot reach $TEST_URL"
		echo "Please make sure that your machine has internet access."
		echo "Test output:"
		echo $TEST_OUT
		exit 1
	fi


In [72]:
TEST_URL="https://pkgs.tailscale.com/"
test_url_resp = requests.get(TEST_URL)

In [86]:
test_url_resp.raise_for_status()
display(HTML(test_url_resp.text))

In [77]:
dir(html)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_charref',
 '_html5',
 '_invalid_charrefs',
 '_invalid_codepoints',
 '_re',
 '_replace_charref',
 'entities',
 'escape',
 'unescape']